In [21]:
import pickle
import pandas as pd
import datetime
import json
import requests
from collections import Counter
from get_tweets import get_tweets

In [2]:
with open('movies_data1.pkl', 'r') as picklefile:
    movies_scraped, movies_skipped = pickle.load(picklefile)

In [3]:
movies = pd.DataFrame(movies_scraped)
movies.dropna(axis=0, subset=['domestic_total_gross'], inplace=True)
movies.sort_values(by='domestic_total_gross', ascending=False, inplace=True)

In [39]:
print len(movies)
print movies[:10]

14682
              BOM_id                                             actors  \
9982       starwars7  John BoyegaDaisy RidleyAdam DriverOscar IsaacA...   
13275         avatar  Sam WorthingtonZoe SaldanaSigourney WeaverMich...   
1772   jurassicpark4  Nick RobinsonOmar SyChris PrattBryce Dallas Ho...   
2616      avengers11  Robert Downey, Jr.Chris HemsworthChris EvansJe...   
7211         titanic  Leonardo DiCaprioKate WinsletBilly ZaneKathy B...   
9173      darkknight  Christian BaleHeath LedgerAaron EckhartMorgan ...   
4547       pixar2015  Ellen DeGeneres (Voice)Albert Brooks (Voice)Ed...   
3565       avengers2  Robert Downey, Jr.Chris HemsworthMark RuffaloC...   
953          batman3  Christian BaleMichael CaineAnne HathawayTom Ha...   
4183          shrek2  Mike Myers (Voice)Cameron Diaz (Voice)Eddie Mu...   

            budget                                   director   distributor  \
9982   245000000.0                                J.J. Abrams   Buena Vista   
13275     

In [22]:
print movies['genre'].unique()
print movies['genre'].value_counts()

[u'Sci-Fi Fantasy' u'Sci-Fi Adventure' u'Action / Adventure' u'Romance'
 u'Animation' u'Action Thriller' u'Period Adventure' u'Sci-Fi Action'
 u'Fantasy' u'Historical Drama' u'Adventure' u'Action' u'Family Adventure'
 u'Sci-Fi Horror' u'Drama' u'Comedy / Drama' u'Horror' u'Family Comedy'
 u'Comedy' u'Sci-Fi Thriller' u'Horror Thriller' u'Sports Drama'
 u'Sci-Fi Comedy' u'Fantasy Comedy' u'Action Drama' u'Romantic Comedy'
 u'Action Comedy' u'Horror Comedy' u'Sci-Fi' u'Fantasy Drama' u'Thriller'
 u'War' u'Period Action' u'Action Horror' u'Historical Epic' u'Western'
 u'Crime Comedy' u'Adventure Comedy' u'Period Drama' u'Musical'
 u'Sports Comedy' u'Drama / Thriller' u'Crime Drama' u'Foreign / Action'
 u'Period Horror' u'Music Drama' u'Western Comedy' u'Documentary'
 u'War Drama' u'Sports Action' u'Period Comedy' u'Crime' u'Action / Crime'
 u'War Romance' u'IMAX' u'Action Fantasy' u'Crime Thriller'
 u'Romantic Thriller' u'Comedy Thriller' u'Family' u'Romantic Adventure'
 u'Concert' u'Fore

In [47]:
time_range = datetime.timedelta(days=45)
maxtweets = 1000

def fetch_tweets(row):
    BOM_id = row['BOM_id']
    movie_title = row['movie_title']
    query = BOM_id + ' OR #' + BOM_id + ' OR ' + movie_title + ' OR #' + movie_title
    date = row['release_date']
    start = date.date().strftime('%Y-%m-%d')
    end = (date.date() + time_range).strftime('%Y-%m-%d')
    get_tweets(filename='data/' + BOM_id, maxtweets=maxtweets, query=query, since=start, until=end)
    print movie_title, start, end

In [48]:
for _, row in movies[:5].iterrows():
    fetch_tweets(row)
    
for _, row in movies[-5005:-5000].iterrows():
    fetch_tweets(row)

Done. Output file generated "data/starwars7".
Star Wars: The Force Awakens 2015-12-18 2016-02-01
Done. Output file generated "data/avatar".
Avatar 2009-12-18 2010-02-01
Done. Output file generated "data/jurassicpark4".
Jurassic World 2015-06-12 2015-07-27
Done. Output file generated "data/avengers11".
Marvel's The Avengers 2012-05-04 2012-06-18
Done. Output file generated "data/titanic".
Titanic 1997-12-19 1998-02-02
Done. Output file generated "data/geniuswithin".
Genius Within: The Inner Life of Glenn Gould 2010-09-10 2010-10-25
Done. Output file generated "data/legendof1900".
The Legend of 1900 1999-10-29 1999-12-13
Done. Output file generated "data/misterlonely".
Mister Lonely 2008-05-02 2008-06-16
Done. Output file generated "data/online".
On Line 2003-06-27 2003-08-11
Done. Output file generated "data/stillwalking".
Still Walking 2009-08-28 2009-10-12


In [58]:
fetched_movie = {}

for BOM_id in movies['BOM_id'][:5]:
    print BOM_id
    fetched_movie[BOM_id] = pd.read_csv('data/' + BOM_id, sep=';')

for BOM_id in movies['BOM_id'][-5005:-5000]:
    print BOM_id
    fetched_movie[BOM_id] = pd.read_csv('data/' + BOM_id, sep=';')

starwars7
avatar
jurassicpark4
avengers11
titanic
geniuswithin
legendof1900
misterlonely
online
stillwalking


In [61]:
def sentiment(text):
    response = requests.post('http://text-processing.com/api/sentiment/', data={'text': text})
    if response.status_code != 200:
        print response.status_code
        print response.content
        print 'failed: {}'.format(text)
        return
    sentiment = json.loads(response.content)['label']
    if sentiment == 'pos':
        return 1
    elif sentiment == 'neg':
        return -1
    else:
        return 0

In [ ]:
for BOM_id, movie in fetched_movie.items():
    sentiments = [sentiment(text) for text in movie['text']]
    movie = movie.assign(sentiment=sentiments)
    movie.to_csv('data/' + BOM_id, sep=';')
    print BOM_id

In [ ]:
for title, movie in fetched_movie.items():
    print title, " : ", Counter(movie['sentiment'])